In [1]:
# http://www.robesafe.uah.es/personal/eduardo.romera/uah-driveset/

# http://www.robesafe.uah.es/personal/eduardo.romera/pdfs/Romera16itsc.pdf
# pdf contains description of data (what each column means)


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

import os



import folium
from geopy.distance import distance
from datetime import datetime

from ride_parser import UAHRideParser, RealRideParser
from app_data_parser import get_data_from_app, generate_acc_from_vel
from PIL import Image

import matplotlib.dates as mdates

In [2]:
def display_velocity_graph(vel_df):
    #  https://stackoverflow.com/questions/75566545/problem-with-matplotlib-date-formatting-and-conversion-from-epoch-time-overlowe
	plt.rcParams['date.converter'] = 'concise'

	fig, axs = plt.subplots(1, 1, figsize=(6.4, 3), layout='constrained')
    # common to all three:
    # for ax in axs:
	# ax[1].plot(time.astype('datetime64[s]'), y)

	# ax.plot('date', 'adj_close', data=data)
	plt.plot("timestamp", "SPEED", data=vel_df)
	# Major ticks every half year, minor ticks every second,
	# axs.xaxis.set_major_locator(mdates.MinuteLocator(byminute=range(60)))
	# axs.xaxis.set_minor_locator(mdates.MinuteLocator())
	axs.grid(True)
	axs.set_ylabel("Velocidade")
	axs.set_xlabel("Horário UTC")

	# ax = axs[0]
	first_timestamp = vel_df.iloc[0]["timestamp"]
	record_day_month_year_string = datetime.utcfromtimestamp(first_timestamp).strftime("%d/%m/%Y")
	axs.set_title("Captura " + record_day_month_year_string)

	axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
	# Rotates and right-aligns the x labels so they don't crowd each other.
	for label in axs.get_xticklabels(which='major'):
		label.set(rotation=30, horizontalalignment='right')

	plt.show()

In [3]:
ride_root_dir = os.path.join("RealData")

real_ride = RealRideParser(ride_root_dir)

latitudes = real_ride.gps_df["lat"]
longitudes = real_ride.gps_df["long"]

qtty_data = len(latitudes)

lat_longs = [(latitudes.iloc[i], longitudes.iloc[i]) for i in range(qtty_data)]

real_ride.accelerometer_df.head(3)

,timestamp,acc_x,acc_y,acc_z,filtered_acc_x,filtered_acc_y,filtered_acc_z,acc_resultant
0,1.699029e+09,0.36,6.04,7.54,0.36,6.04,7.54,9.667616
1,1.699029e+09,0.34,5.97,7.65,0.34,5.97,7.65,9.709737
2,1.699029e+09,0.38,6.14,7.58,0.38,6.14,7.58,9.762192


In [4]:
app_data = get_data_from_app()

velocities = app_data["SPEED"]

display_velocity_graph(velocities)

C:\Users\Tutubarao\AppData\Local\Temp\ipykernel_16680\1948564346.py:21: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  record_day_month_year_string = datetime.utcfromtimestamp(first_timestamp).strftime("%d/%m/%Y")


OverflowError: int too big to convert

Error in callback <function _draw_all_if_interactive at 0x000001574EF56160> (for post_execute), with arguments args (),kwargs {}:


OverflowError: int too big to convert

OverflowError: int too big to convert

<Figure size 640x300 with 1 Axes>

In [ ]:
ride_root_dir = os.path.join("RealData")

real_ride = RealRideParser(ride_root_dir)

# real_ride.gps_df.dtypes
# ride.gps_df
# ride.accelerometer_df

real_ride.get_acc_stats()
real_ride.gps_df
velocity_df = real_ride.create_velocity_from_gps_df()
display_velocity_graph(velocity_df)
# velocity_df.plot(x="timestamp", y="velocity")

In [ ]:
# app_data = get_data_from_app()

# velocities = app_data["SPEED"]
# display_velocity_graph(velocities)

# print(app_data["SPEED"])

In [ ]:

display_velocity_graph(generate_acc_from_vel(velocities))


In [ ]:
start_lat_long = lat_longs[0]
end_lat_long = lat_longs[-1]

mean_lat = latitudes.mean()
mean_long = longitudes.mean()

start_lat_long, end_lat_long, qtty_data

In [ ]:
map = folium.Map(location=[mean_lat, mean_long], zoom_start=14, control_scale=True)

folium.Marker(start_lat_long, popup="start").add_to(map)
folium.Marker(end_lat_long, popup="end").add_to(map)

In [ ]:
for i in range(qtty_data - 1):
	location_1 = lat_longs[i]
	location_2 = lat_longs[i + 1]

	folium.PolyLine([location_1, location_2],
					color='red',
					weight=5,
					opacity=0.4).add_to(map)

map

In [ ]:
accelerometer_df = real_ride.accelerometer_df
last_timestamp = accelerometer_df.timestamp.iloc[-1] # .timestamp()
delta_time_size = 100 	# 100 s
frame_granularity = 10 	# 50 s
qtty_frames = (last_timestamp - delta_time_size) // frame_granularity

filtered_accelerations_df = accelerometer_df

print("last_timestamp", last_timestamp)
print("qtty_frames", qtty_frames)
print("last_timestamp", last_timestamp)

qtty_frames = min(qtty_frames, 100)

# foo_for_real_data(accelerometer_df, frame_granularity, delta_time_size, 0)

for i in range(10): #qtty_frames):
    real_ride.foo_for_real_data(frame_granularity, delta_time_size, i)


Objetivos:
- [] Gerar mapa com a rota feita por cada um dos motoristas
- [] Sincronizar vídeo com eventos de aceleração

In [ ]:
# ride_root_dir = os.path.join("UAH-DRIVESET-v1", "D1", "20151110175712-16km-D1-NORMAL1-SECONDARY")
ride_root_dir = os.path.join("UAH-DRIVESET-v1", "D1", "20151111125233-24km-D1-AGGRESSIVE-MOTORWAY")

ride = UAHRideParser(ride_root_dir)

# col_names = [str(i) for i in range(13)]
# col_names = ["timestamp", "speed", "lat", "long", "altitude", "vert accuracy", "horiz accuracy", "course", "difcourse", "?1", "?2", "?3", "?4"]
# pd.read_csv(gps_file_path, sep=" ", names=col_names)

ride.gps_df
ride.accelerometer_df

In [ ]:
accelerometer_df = ride.accelerometer_df
oldest_timestamp = int(accelerometer_df.timestamp.iloc[-1])
delta_time_size = 20
frame_granularity = 1
qtty_frames = (oldest_timestamp - delta_time_size) // frame_granularity

print("qtty_frames", qtty_frames)
print("oldest_timestamp", oldest_timestamp)

for i in range(10): #qtty_frames):
    with ProcessPoolExecutor(max_workers=10) as executor:
        future = executor.submit(ride.foo, accelerometer_df, frame_granularity, delta_time_size, i)

    # print(future.result())

# plt.show()
# RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.

In [ ]:
fig, axs = plt.subplots(ncols=1, nrows=4)

accelerometer_df = ride.accelerometer_df
accelerometer_df = accelerometer_df[accelerometer_df.timestamp < 20]
filtered_acc_x = accelerometer_df["filtered_acc_x"]
filtered_acc_y = accelerometer_df["filtered_acc_y"]
filtered_acc_z = accelerometer_df["filtered_acc_z"]
acc_resultant = accelerometer_df["acc_resultant"]
timestamp = accelerometer_df["timestamp"]

# axs[0].scatter(timestamp, filtered_acc_x, s=0.1)
axs[0].plot(timestamp, filtered_acc_x) #, "-o")
axs[0].set_title("filtered_acc_x")

axs[1].plot(timestamp, filtered_acc_y)
axs[1].set_title("filtered_acc_y")

axs[2].plot(timestamp, filtered_acc_z)
axs[2].set_title("filtered_acc_z")

axs[3].plot(timestamp, acc_resultant)
axs[3].set_title("acc_resultant")

fig.tight_layout()

plt.show()

In [ ]:
imgs = ride.generate_acc_sub_graph()

# im = Image.open(next(imgs))
# im.show(title="My Image")

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.cm as cm
# import matplotlib.animation as animation

# import numpy
# from PIL import Image

# n = 20

# img = [] # some array of images
# for i in range(n):
# 	imarray = numpy.random.rand(100,100,3) * 255
# 	im = Image.fromarray(imarray.astype('uint8')).convert('RGBA')
# 	img.append(im)

# frames = [] # for storing the generated images
# fig = plt.figure()
# for i in range(n):
# 	frames.append([plt.imshow(img[i], cmap=cm.Greys_r,animated=True)])

# ani = animation.ArtistAnimation(fig, frames, blit=True)

# writergif = animation.PillowWriter(fps=1)
# ani.save('movie.gif', writer=writergif)
# plt.show()

In [ ]:
# import io
# from PIL import Image
# import matplotlib.pyplot as plt

# plt.rcParams["figure.figsize"] = [7.50, 3.50]
# plt.rcParams["figure.autolayout"] = True

# plt.figure()
# plt.plot([1, 2])

# img_buf = io.BytesIO()
# plt.savefig(img_buf, format='png')

# im = Image.open(img_buf)
# im.show(title="My Image")

# img_buf.close()